In [1]:
import sympy as sym
import numpy as np
np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})
import matplotlib.pyplot as plt

In [2]:
def neglect_terms(expr, N):
    # Function to neglect terms of order N and higher in a polynomial
    # Split the expression into numerator and denominator
    num, denom = expr.as_numer_denom()
    # Process numerator and denominator to remove terms of order N and higher
    num_neglected = sum(coeff * term for term, coeff in num.as_coefficients_dict().items() if all(pow < N for var, pow in term.as_powers_dict().items()))
    denom_neglected = sum(coeff * term for term, coeff in denom.as_coefficients_dict().items() if all(pow < N for var, pow in term.as_powers_dict().items()))
    # Reconstruct the fraction
    return num_neglected / denom_neglected

def neglect_terms_in_matrix(matrix, N):
    # Process each element in the matrix
    processed_matrix = matrix.applyfunc(lambda x: neglect_terms(x, N))

    return processed_matrix

In [17]:
Φp, Φm, Φp_prime, Φm_prime = sym.symbols('\dot{\Phi}_+, \dot{\Phi}_-, Phiprimedot_+, Phiprimedot_-', real=True, positive=True, nonzero=True)
Cp, Cm, Cc, Cp_prime, Cm_prime, eps = sym.symbols('C_+, C_-, C_C, Cprime_+, Cprime_-, varepsilon', real=True, positive=True, nonzero=True)

In [4]:
Φ_vec = sym.Matrix([Φp, Φm, Φp_prime, Φm_prime])

In [5]:
Φ1 = 1/2*(Φp+Φm)
Φ2 = 1/2*(Φp-Φm)
Φ1_prime = 1/2*(Φp_prime+Φm_prime)
Φ2_prime = 1/2*(Φp_prime-Φm_prime)

In [6]:
# Change of variables from (0,1) to (p,m)
P1 = 0.5 *sym.Matrix([  [1, 1  ,0  ,0 ],
                        [1,-1  ,0  ,0 ],
                        [0, 0  ,1  , 1],
                        [0, 0  ,1  ,-1] ])

## Asymmetric capacitive coupling between two unit cells (only inner nodes) 

In [7]:
sym.expand(Cc*(Φ2-Φ1_prime)**2 )

0.25*C_C*Phiprimedot_+**2 + 0.5*C_C*Phiprimedot_+*Phiprimedot_- - 0.5*C_C*Phiprimedot_+*\dot{\Phi}_+ + 0.5*C_C*Phiprimedot_+*\dot{\Phi}_- + 0.25*C_C*Phiprimedot_-**2 - 0.5*C_C*Phiprimedot_-*\dot{\Phi}_+ + 0.5*C_C*Phiprimedot_-*\dot{\Phi}_- + 0.25*C_C*\dot{\Phi}_+**2 - 0.5*C_C*\dot{\Phi}_+*\dot{\Phi}_- + 0.25*C_C*\dot{\Phi}_-**2

In [8]:
C_mat_01 = sym.Matrix([ [0          ,0          ,0          ,0          ],
                        [0          , Cc        ,-Cc      ,0          ],
                        [0          ,-Cc      , Cc        ,0          ],
                        [0          ,0          ,0          ,0          ] ])

In [9]:
P1.T * C_mat_01 * P1

Matrix([
[ 0.25*C_C, -0.25*C_C, -0.25*C_C, -0.25*C_C],
[-0.25*C_C,  0.25*C_C,  0.25*C_C,  0.25*C_C],
[-0.25*C_C,  0.25*C_C,  0.25*C_C,  0.25*C_C],
[-0.25*C_C,  0.25*C_C,  0.25*C_C,  0.25*C_C]])

In [10]:
C_mat = sym.Matrix([[ Cc/4    ,-Cc/4    ,-Cc/4    ,-Cc/4    ],
                    [-Cc/4    , Cc/4    , Cc/4    , Cc/4    ],
                    [-Cc/4    , Cc/4    , Cc/4    ,+Cc/4    ],
                    [-Cc/4    , Cc/4    ,+Cc/4    , Cc/4    ] ])

In [11]:
sym.expand( Φ_vec.T * C_mat * Φ_vec ) 

Matrix([[C_C*Phiprimedot_+**2/4 + C_C*Phiprimedot_+*Phiprimedot_-/2 - C_C*Phiprimedot_+*\dot{\Phi}_+/2 + C_C*Phiprimedot_+*\dot{\Phi}_-/2 + C_C*Phiprimedot_-**2/4 - C_C*Phiprimedot_-*\dot{\Phi}_+/2 + C_C*Phiprimedot_-*\dot{\Phi}_-/2 + C_C*\dot{\Phi}_+**2/4 - C_C*\dot{\Phi}_+*\dot{\Phi}_-/2 + C_C*\dot{\Phi}_-**2/4]])

In [12]:
sym.simplify( (Φ_vec.T * C_mat * Φ_vec)[0] - (Cc*(Φ2-Φ1_prime)**2))

0

## Asymmetric capacitive coupling between two unit cells (only inner nodes) with outer node compensation

In [13]:
sym.expand(Cc*(Φ2-Φ1_prime)**2 + Cc*(Φ1)**2 + Cc*(Φ2_prime)**2)

0.5*C_C*Phiprimedot_+**2 - 0.5*C_C*Phiprimedot_+*\dot{\Phi}_+ + 0.5*C_C*Phiprimedot_+*\dot{\Phi}_- + 0.5*C_C*Phiprimedot_-**2 - 0.5*C_C*Phiprimedot_-*\dot{\Phi}_+ + 0.5*C_C*Phiprimedot_-*\dot{\Phi}_- + 0.5*C_C*\dot{\Phi}_+**2 + 0.5*C_C*\dot{\Phi}_-**2

In [14]:
C_mat = sym.Matrix([[ Cc/2    , 0       ,-Cc/4    ,-Cc/4    ],
                    [ 0       , Cc/2    , Cc/4    , Cc/4    ],
                    [-Cc/4    , Cc/4    , Cc/2    , 0       ],
                    [-Cc/4    , Cc/4    , 0       , Cc/2    ] ])

In [15]:
sym.expand( Φ_vec.T * C_mat * Φ_vec ) 

Matrix([[C_C*Phiprimedot_+**2/2 - C_C*Phiprimedot_+*\dot{\Phi}_+/2 + C_C*Phiprimedot_+*\dot{\Phi}_-/2 + C_C*Phiprimedot_-**2/2 - C_C*Phiprimedot_-*\dot{\Phi}_+/2 + C_C*Phiprimedot_-*\dot{\Phi}_-/2 + C_C*\dot{\Phi}_+**2/2 + C_C*\dot{\Phi}_-**2/2]])

In [16]:
sym.simplify( (Φ_vec.T * C_mat * Φ_vec)[0] - (Cc*(Φ2-Φ1_prime)**2 + Cc*(Φ1)**2 + Cc*(Φ2_prime)**2))

0

In [20]:
C_mat = sym.Matrix([[ Cp    , 0       ,-eps   ,-eps    ],
                    [ 0       , Cm    , eps   , eps   ],
                    [-eps    , eps   , Cp   , 0       ],
                    [-eps   , eps   , 0       , Cm   ] ])

D = sym.Matrix([[ Cp    , 0       ,0  ,0    ],
                    [ 0       , Cm    , 0  ,0  ],
                    [0    , 0  , Cp   , 0       ],
                    [0   , 0  , 0       , Cm   ] ])
R = sym.Matrix([[ 0  , 0       ,-eps   ,-eps    ],
                    [ 0       , 0  , eps   , eps   ],
                    [-eps    , eps   , 0  , 0       ],
                    [-eps   , eps   , 0       ,0  ] ])

In [22]:
D**-1 

Matrix([
[1/C_+,     0,     0,     0],
[    0, 1/C_-,     0,     0],
[    0,     0, 1/C_+,     0],
[    0,     0,     0, 1/C_-]])

In [27]:
-D**-1 *R  * D**-1

Matrix([
[                   0,                     0,     varepsilon/C_+**2, varepsilon/(C_+*C_-)],
[                   0,                     0, -varepsilon/(C_+*C_-),   -varepsilon/C_-**2],
[   varepsilon/C_+**2, -varepsilon/(C_+*C_-),                     0,                    0],
[varepsilon/(C_+*C_-),    -varepsilon/C_-**2,                     0,                    0]])

## Symmetric capacitive coupling between two unit cells (inner and outter nodes)

In [18]:
sym.expand(Cc*(Φ2-Φ1_prime)**2 + Cc*(Φ1-Φ2_prime)**2)

0.5*C_C*Phiprimedot_+**2 - 1.0*C_C*Phiprimedot_+*\dot{\Phi}_+ + 0.5*C_C*Phiprimedot_-**2 + 1.0*C_C*Phiprimedot_-*\dot{\Phi}_- + 0.5*C_C*\dot{\Phi}_+**2 + 0.5*C_C*\dot{\Phi}_-**2

In [19]:
C_mat = sym.Matrix([[ Cc/2    , 0       ,-Cc/2    , 0       ],
                    [ 0       , Cc/2    , 0       , Cc/2    ],
                    [-Cc/2    , 0       , Cc/2    , 0       ],
                    [ 0       , Cc/2    , 0       , Cc/2    ] ])

In [20]:
sym.simplify( (Φ_vec.T * C_mat * Φ_vec)[0] - (Cc*(Φ2-Φ1_prime)**2 + Cc*(Φ1-Φ2_prime)**2))

0

In [21]:
LR, LF, Lr, Lq, Δ = sym.symbols('L_R, L_F, L_r, L_q, Delta', real=True, positive=True, nonzero=True)

In [22]:
l2 = Lq*(Lq+4*Lr) - 4 * Δ ** 2

In [23]:
LR  = l2 / Lq
LF  = l2 / (Lq + 4*Lr)

In [24]:
LR.subs({Lq:15, Lr:5, Δ:1}), LF.subs({Lq:15, Lr:5, Δ:1}), 

(521/15, 521/35)

In [ ]:
LR_eq = sym.Eq(LR, l2 / Lq)
LF_eq = sym.Eq(LF, l2 / (Lq + 4*Lr))

# We need to solve these equations to express Lq and Lr in terms of LF, LR, and Delta
solution = sym.solve((LR_eq, LF_eq), (Lq, Lr))

In [106]:
sym.simplify( solution[1][0])

sqrt(L_F)*sqrt(16*Delta**2 + L_F*L_R)/(2*sqrt(L_R)) + L_F/2

In [107]:
sym.simplify( solution[1][1])

-(L_F - L_R)*(sqrt(L_F)*sqrt(L_R) + sqrt(16*Delta**2 + L_F*L_R))/(8*sqrt(L_F)*sqrt(L_R))

# Below this line lies the past, above the future

In [ ]:
C_R, C_F, C_C = sym.symbols('C_R, C_F, C_C', real=True, positive=True, nonzero=True)

In [5]:
C_mat = sym.Matrix([[C_R+C_C/2 , 0         , -C_C/2    , +C_C/2     ],
                    [0         , C_F+C_C/2 , -C_C/2    , +C_C/2     ],
                    [-C_C/2    , +C_C/2    , C_R+C_C/2 , 0          ],
                    [-C_C/2    , +C_C/2    , 0         , C_F+C_C/2  ] ])

In [14]:
C_mat = sym.Matrix([[C_R+C_C/2 , 0         , -C_C/2    , 0         ],
                     [0        , C_F+C_C/2 , 0         , +C_C/2    ],
                     [-C_C/2   , 0         , C_R+C_C/2 , 0         ],
                     [0        , +C_C/2    , 0         , C_F+C_C/2 ] ])

In [38]:
C_inv = C_mat**-1

In [39]:
sym.simplify(C_inv)

Matrix([
[2*(C_C**3 + 6*C_C**2*C_F + 4*C_C*C_F**2 + 8*C_C*C_F*C_R + 8*C_F**2*C_R)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2),                                                   4*C_C**2*(-C_F + C_R)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2),                                   2*C_C*(C_C**2 + 4*C_C*C_F + 4*C_F**2)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2),                     2*C_C*(-C_C**2 - 2*C_C*C_F - 2*C_C*C_R - 4*C_F*C_R)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2)],
[                                                   4*C_C**2*(C_F - C_R)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2), 2*(C_C**3 + 6*C_C**2*C_R + 8*C_C*

In [9]:
Csh = 15
C   = 15
CJ  = 3
Cc  = 0.1

In [10]:
C_R, C_F, C_C = sym.symbols('C_R, C_F, C_C', real=True, positive=True, nonzero=True)
C_inv_sym_to_num = np.array( C_inv.subs({C_C:Cc, C_R:C/2, C_F:C/2 + Csh + CJ}).evalf() )

NameError: name 'C_inv' is not defined

In [16]:
C_R = C/2
C_C = Cc
C_F = C/2 + Csh + CJ

𐌲_R = C / (C_R+C_C/2)
𐌲_F = C / (C_F+C_C/2)
ε = C_C/2 / C

C_inv_approx = 1/C * np.array([[ 𐌲_R    ,   0       ,  ε*𐌲_R**2,  -ε*𐌲_R**2],
                               [0       ,   𐌲_F     ,  ε*𐌲_F**2,  -ε*𐌲_F**2],
                               [ε*𐌲_R**2,  -ε*𐌲_R**2,   𐌲_R    ,   0       ],
                               [ε*𐌲_F**2,  -ε*𐌲_F**2,   0      ,   𐌲_F     ]])

# C_inv_approx = 1/C * np.array([[ 𐌲_R    ,   0       ,  ε*𐌲_R**2,   0         ],
#                                [0       ,   𐌲_F     ,      0  ,  -ε*𐌲_F**2  ],
#                                [ε*𐌲_R**2,   0       ,   𐌲_R    ,   0         ],
#                                [0       ,  -ε*𐌲_F**2,   0      ,   𐌲_F       ]])

In [17]:
C_F / C_R

3.4

In [18]:
C_num = np.array  ([[C_R+C_C/2, 0      , -C_C/2 , +C_C/2   ],
                    [0      , C_F+C_C/2, -C_C/2 , +C_C/2   ],
                    [-C_C/2 , +C_C/2 , C_R+C_C/2, 0        ],
                    [-C_C/2 , +C_C/2 , 0      , C_F+C_C/2] ])
# C_num = np.array  ([[C_R+C_C/2, 0      , -C_C/2 ,0       ],
#                     [0      , C_F+C_C/2, 0      , +C_C/2 ],
#                     [-C_C/2 , 0      , C_R+C_C/2, 0      ],
#                     [0      , +C_C/2 , 0      , C_F+C_C/2] ])

In [19]:
np.linalg.inv(C_num)

array([[1.325e-01, -1.209e-06, 8.772e-04, -2.592e-04],
       [1.209e-06, 3.914e-02, 2.592e-04, -7.659e-05],
       [8.772e-04, -2.592e-04, 1.325e-01, -1.209e-06],
       [2.592e-04, -7.659e-05, 1.209e-06, 3.914e-02]])

In [20]:
(1 + (1 - C_R/C) + (1 - C_R/C)**2)/C

0.11666666666666667

In [21]:
(1 + (1 - C_F/C) + (1 - C_F/C)**2)/C

0.05266666666666667

In [23]:
-2*ε*(1-C_R/C)**2 /C

-0.00011111111111111112

In [27]:
(ε*(1-C_R/C) + ε*(1-C_F/C) )/C

-4.444444444444445e-05

In [222]:
C_inv_approx

array([[1.325e-01, 0.000e+00, 8.772e-04, -8.772e-04],
       [0.000e+00, 1.325e-01, 8.772e-04, -8.772e-04],
       [8.772e-04, -8.772e-04, 1.325e-01, 0.000e+00],
       [8.772e-04, -8.772e-04, 0.000e+00, 1.325e-01]])

In [214]:
ε**2

1.1111111111111113e-05

In [217]:
0.001**2

1e-06

In [10]:
# Define the symbols
alpha= sym.symbols('\\alpha', real=True, positive = True, nonzero=True)
N=6
C_mat = sym.Matrix(N,N, lambda i, j: -1 if abs(i-j) == 2 else 0)
C_mat**2

Matrix([
[1, 0, 0, 0, 1, 0],
[0, 1, 0, 0, 0, 1],
[0, 0, 2, 0, 0, 0],
[0, 0, 0, 2, 0, 0],
[1, 0, 0, 0, 1, 0],
[0, 1, 0, 0, 0, 1]])

In [52]:
# Define the symbols
alpha= sym.symbols('\\alpha', real=True, positive = True, nonzero=True)
N=6
C_mat = sym.Matrix(N,N, lambda i, j: 1+alpha if i == j else -alpha if abs(i-j) == 2 else -alpha if ((i==N-1 and j==0) or (i==0 and j==N-1) )  else 0)
# C_mat = sym.Matrix(N,N, lambda i, j: 1+alpha if i == j else -alpha if abs(i-j) == 2 else -alpha if abs(i-j) == 1 else -alpha if ((i==N-1 and j==0) or (i==0 and j==N-1) )  else 0)
# C_mat = sym.Matrix(N,N, lambda i, j: 1+alpha if i == j else -alpha if abs(i-j) == 2  else 0)

C_mat = sym.Matrix([[1+alpha, -alpha  , alpha , alpha  , 0,  0],
                    [-alpha , 2+alpha, -alpha , -alpha  , 0, 0 ],
                    [alpha  , -alpha ,3+2*alpha , 0      , alpha, alpha ],
                    [alpha  , -alpha , 0      , 1+2*alpha, -alpha, -alpha],
                    [0      ,  0     , alpha  , -alpha    , 4+alpha, alpha],
                    [0      , 0      , alpha  , -alpha    , alpha, 0.7+alpha] ])

# C_mat = sym.Matrix([[1, -alpha  , alpha , alpha  , 0,  0],
#                     [-alpha , 1, -alpha , -alpha  , 0, 0 ],
#                     [alpha  , -alpha ,1 , 0      , alpha, alpha ],
#                     [alpha  , -alpha , 0      , 1, -alpha, -alpha],
#                     [0      ,  0     , alpha  , -alpha    , 1, alpha],
#                     [0      , 0      , alpha  , -alpha    , alpha, 1] ])

C_mat

Matrix([
[\alpha + 1,    -\alpha,       \alpha,       \alpha,          0,            0],
[   -\alpha, \alpha + 2,      -\alpha,      -\alpha,          0,            0],
[    \alpha,    -\alpha, 2*\alpha + 3,            0,     \alpha,       \alpha],
[    \alpha,    -\alpha,            0, 2*\alpha + 1,    -\alpha,      -\alpha],
[         0,          0,       \alpha,      -\alpha, \alpha + 4,       \alpha],
[         0,          0,       \alpha,      -\alpha,     \alpha, \alpha + 0.7]])

In [55]:
np.linalg.inv(np.array((C_mat.subs(alpha,1)), dtype='float'))

array([[7.314e-01, 1.343e-01, -1.044e-01, -2.240e-01, -1.116e-02, -6.377e-02],
       [1.343e-01, 4.328e-01, 5.221e-02, 1.120e-01, 5.580e-03, 3.189e-02],
       [-1.044e-01, 5.221e-02, 2.650e-01, -3.986e-03, -2.511e-02, -1.435e-01],
       [-2.240e-01, 1.120e-01, -3.986e-03, 5.640e-01, 5.301e-02, 3.029e-01],
       [-1.116e-02, 5.580e-03, -2.511e-02, 5.301e-02, 2.340e-01, -9.167e-02],
       [-6.377e-02, 3.189e-02, -1.435e-01, 3.029e-01, -9.167e-02, 9.047e-01]])

In [53]:
C_inv = C_mat **-1
# C_inv = sym.simplify(C_inv)

In [54]:
C_inv

Matrix([
[(65143644120.0*\alpha**34 + 2489381574660.0*\alpha**33 + 45040794026269.5*\alpha**32 + 514614924155669.0*\alpha**31 + 4.17837383544339e+15*\alpha**30 + 2.57230393375125e+16*\alpha**29 + 1.25090356887141e+17*\alpha**28 + 4.94255247740483e+17*\alpha**27 + 1.6195304834827e+18*\alpha**26 + 4.46912739250521e+18*\alpha**25 + 1.05100208031684e+19*\alpha**24 + 2.12597779092207e+19*\alpha**23 + 3.72611259748985e+19*\alpha**22 + 5.69091143630759e+19*\alpha**21 + 7.60788289781084e+19*\alpha**20 + 8.93213842062639e+19*\alpha**19 + 9.23197339876618e+19*\alpha**18 + 8.41282383115429e+19*\alpha**17 + 6.76399416519681e+19*\alpha**16 + 4.79776373192268e+19*\alpha**15 + 2.99956409866769e+19*\alpha**14 + 1.65006695899311e+19*\alpha**13 + 7.96537610141542e+18*\alpha**12 + 3.36164599371037e+18*\alpha**11 + 1.23418458868363e+18*\alpha**10 + 3.91629834683991e+17*\alpha**9 + 1.06510735277517e+17*\alpha**8 + 2.45581034835782e+16*\alpha**7 + 4.73196722439767e+15*\alpha**6 + 747360502225920.0*\alpha**5

In [152]:
neglect_terms_in_matrix(C_inv,2)

Matrix([
[(8*\alpha + 4)/(12*\alpha + 4),                              0,       4*\alpha/(12*\alpha + 4),      -2*\alpha/(12*\alpha + 4)],
[                             0, (5*\alpha + 2)/(12*\alpha + 4),       2*\alpha/(12*\alpha + 4),        -\alpha/(12*\alpha + 4)],
[      4*\alpha/(12*\alpha + 4),      -2*\alpha/(12*\alpha + 4), (8*\alpha + 4)/(12*\alpha + 4),                              0],
[      2*\alpha/(12*\alpha + 4),        -\alpha/(12*\alpha + 4),                              0, (5*\alpha + 2)/(12*\alpha + 4)]])

In [9]:
U_0   = sym.polys.orthopolys.chebyshevu_poly(0,   x=(alpha+1)/(2*alpha)).simplify()
U_N   = sym.polys.orthopolys.chebyshevu_poly(N,   x=(alpha+1)/(2*alpha)).simplify()
U_Np1 = sym.polys.orthopolys.chebyshevu_poly(N+1, x=(alpha+1)/(2*alpha)).simplify()
U_Np2 = sym.polys.orthopolys.chebyshevu_poly(N+2, x=(alpha+1)/(2*alpha)).simplify()

In [17]:
C_inv[0,0]

(-2*\alpha**4 + \alpha**3 + 7*\alpha**2 + 5*\alpha + 1)/(\alpha**6 - 2*\alpha**5 - 9*\alpha**4 + 10*\alpha**2 + 6*\alpha + 1)

In [10]:
C_inv_diag = U_Np2**-1 * alpha ** -1 * U_0 * U_Np1
C_inv_diag.expand().simplify()

(\alpha**7 + 3*\alpha**6 - 9*\alpha**5 - 15*\alpha**4 + 5*\alpha**3 + 15*\alpha**2 + 7*\alpha + 1)/(6*\alpha**7 + 3*\alpha**6 - 24*\alpha**5 - 20*\alpha**4 + 14*\alpha**3 + 21*\alpha**2 + 8*\alpha + 1)

In [11]:
C_inv_sub_diag = U_Np2**-1 * alpha ** -1 * U_0 * U_N
C_inv_sub_diag.expand().simplify()

\alpha*(\alpha**6 - 2*\alpha**5 - 9*\alpha**4 + 10*\alpha**2 + 6*\alpha + 1)/(6*\alpha**7 + 3*\alpha**6 - 24*\alpha**5 - 20*\alpha**4 + 14*\alpha**3 + 21*\alpha**2 + 8*\alpha + 1)

In [6]:
# Define the symbols
C_R, C_F, C_C = sym.symbols('C_R, C_F, C_C', real=True, positive=True, nonzero=True)

# Create a 6x6 matrix based on the given pattern
C_mat = sym.Matrix(4,4, lambda i, j: C_R+C_C/2 if (i == j and np.mod(i,2)==0) else C_F+C_C/2 if (i == j and np.mod(i,2)!=0)  else -C_C/2 if abs(i-j) == 2 else 0)
C_mat[1,-1] *=-1
C_mat[-1,1] *=-1
C_mat

Matrix([
[C_C/2 + C_R,           0,      -C_C/2,           0],
[          0, C_C/2 + C_F,           0,       C_C/2],
[     -C_C/2,           0, C_C/2 + C_R,           0],
[          0,       C_C/2,           0, C_C/2 + C_F]])

In [7]:
C_inv = C_mat **-1

In [8]:
sym.simplify(C_inv)

Matrix([
[(C_C/2 + C_R)/(C_R*(C_C + C_R)),                               0,         C_C/(2*C_R*(C_C + C_R)),                               0],
[                              0, (C_C/2 + C_F)/(C_F*(C_C + C_F)),                               0,        -C_C/(2*C_F*(C_C + C_F))],
[        C_C/(2*C_R*(C_C + C_R)),                               0, (C_C/2 + C_R)/(C_R*(C_C + C_R)),                               0],
[                              0,        -C_C/(2*C_F*(C_C + C_F)),                               0, (C_C/2 + C_F)/(C_F*(C_C + C_F))]])

In [30]:
sym.print_latex(sym.simplify(C_inv))

\left[\begin{matrix}\frac{\frac{C_{C}}{2} + C_{R}}{C_{R} \left(C_{C} + C_{R}\right)} & 0 & \frac{C_{C}}{2 C_{R} \left(C_{C} + C_{R}\right)} & 0\\0 & \frac{\frac{C_{C}}{2} + C_{F}}{C_{F} \left(C_{C} + C_{F}\right)} & 0 & \frac{C_{C}}{2 C_{F} \left(C_{C} + C_{F}\right)}\\\frac{C_{C}}{2 C_{R} \left(C_{C} + C_{R}\right)} & 0 & \frac{\frac{C_{C}}{2} + C_{R}}{C_{R} \left(C_{C} + C_{R}\right)} & 0\\0 & \frac{C_{C}}{2 C_{F} \left(C_{C} + C_{F}\right)} & 0 & \frac{\frac{C_{C}}{2} + C_{F}}{C_{F} \left(C_{C} + C_{F}\right)}\end{matrix}\right]


In [18]:
# Define the symbols
C_R, C_F, C_C = sym.symbols('C_R, C_F, C_C', real=True, positive=True, nonzero=True)

# Create a 6x6 matrix based on the given pattern
C_mat = sym.Matrix(10,10, lambda i, j: C_R+C_C/2 if (i == j and np.mod(i,2)==0) else C_F+C_C/2 if (i == j and np.mod(i,2)!=0)  else -C_C/2 if abs(i-j) == 2 else 0)

C_mat

Matrix([
[C_C/2 + C_R,           0,      -C_C/2,           0,           0,           0,           0,           0,           0,           0],
[          0, C_C/2 + C_F,           0,      -C_C/2,           0,           0,           0,           0,           0,           0],
[     -C_C/2,           0, C_C/2 + C_R,           0,      -C_C/2,           0,           0,           0,           0,           0],
[          0,      -C_C/2,           0, C_C/2 + C_F,           0,      -C_C/2,           0,           0,           0,           0],
[          0,           0,      -C_C/2,           0, C_C/2 + C_R,           0,      -C_C/2,           0,           0,           0],
[          0,           0,           0,      -C_C/2,           0, C_C/2 + C_F,           0,      -C_C/2,           0,           0],
[          0,           0,           0,           0,      -C_C/2,           0, C_C/2 + C_R,           0,      -C_C/2,           0],
[          0,           0,           0,           0,           0,  

In [19]:
C_inv = C_mat **-1

In [20]:
sym.simplify(C_inv)

Matrix([
[(-C_C**4/4 - C_C**3*C_R + 3*C_C**2*C_R**2 + 8*C_C*C_R**3 + 4*C_R**4)/(C_R*(-C_C**4 - C_C**3*C_R + 6*C_C**2*C_R**2 + 10*C_C*C_R**3 + 4*C_R**4)),                                                                                                                                              0,                C_C*(-C_C**2 + 4*C_C*C_R + 4*C_R**2)/(4*C_R*(-C_C**3 + C_C**2*C_R + 4*C_C*C_R**2 + 2*C_R**3)),                                                                                                            0,            C_C**2/(-C_C**3 + 6*C_C*C_R**2 + 4*C_R**3),                                                     0,                                              C_C**3/(4*C_R*(-C_C**3 + C_C**2*C_R + 4*C_C*C_R**2 + 2*C_R**3)),                                                                                                            0,                                                             C_C**4/(4*C_R*(-C_C**4 - C_C**3*C_R + 6*C_C**2*C_R**2 + 10*C_C*C_R**3 + 4*C_R**4)),         